# Проверка структуры сайта в целом. Русскоязычные страницы.
Общее время: около 35-40 минут (в зависимости от стабильности интернет соединения)

Подсчет количества знаков в **каждой** полученной ссылке: 2 часа 45 минут и более  

Производится переход по каждой ссылке (то есть по всем ссылкам на академических руководителей всех ОП, на всех менеджеров и тд), чтобы удостовериться, что страницы заполнены. Поэтому этот подсчет длится дольше всех.
 
**Партнеры** встречаются и в Общих разделах и в разделе Главное. В Общих: названия, в Главном: ссылки.  
**О программе** встречается и в Общих разделах и в разделе Главное. В Общих: количество символов, в Главном: ссылки.

In [1]:
import requests as rq
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time

start_time = time.time()

### Создадим результирующую таблицу и заполним основную информацию о программах

In [2]:
result = pd.DataFrame(columns=['Наименование ОП', 'Уровень обучения',
                               'Подразделение', 'Ссылка на сайт'])

In [3]:
def soup_d(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
def main():
    url = 'https://www.hse.ru/education/bachelor/'
    html = rq.get(url).text
    soup = soup_d(html)
    text = soup.findAll(
        'div', {'class': 'b-row__item b-row__item--6 b-row__item--t12'})
    quantity = range(len(text))

    # Выгрузим ссылки на все ОП

    urls = [text[i].contents[0].get('href') for i in quantity]

    # Заменим нестандартные страницы школы дизайна
    urls[urls.index('https://design.hse.ru/ba/program/design')
         ] = 'https://www.hse.ru/ba/design/'
    urls[urls.index('https://design.hse.ru/ba/program/fashion')
         ] = 'https://www.hse.ru/ba/moda/'
    urls[urls.index('https://art.hse.ru/ba')
         ] = 'https://www.hse.ru/ba/art/'
    urls[urls.index('http://design.hse.ru/mag-design')
         ] = 'https://www.hse.ru/ma/cd/'
    urls[urls.index('https://design.hse.ru/ma/program/interior')
         ] = 'https://www.hse.ru/ma/idesign/'
    urls[urls.index('http://design.hse.ru/mag-comm')
         ] = 'https://www.hse.ru/ma/designcom/'
    urls[urls.index('http://design.hse.ru/mag-fashion')
         ] = 'https://www.hse.ru/ma/fashion/'
    urls[urls.index('http://art.hse.ru/ma')
         ] = 'https://www.hse.ru/ma/contart/'
    urls[urls.index('http://design.hse.ru/pages/184')
         ] = 'https://www.hse.ru/ma/moddis/'

    # Удалим еще не открытую программу
    urls.remove('https://www.hse.ru/ma/digitalpm/')

    result['Ссылка на сайт'] = urls

    # Названия ОП
    names = [text[i].contents[0].contents[0] for i in quantity]
    names.remove('Управление цифровым продуктом')  # Не открыта
    result['Наименование ОП'] = names

    # Уровень обучения
    bac_q = names.index('Фундаментальная и прикладная лингвистика')+1
    mag_q = len(names)-bac_q
    d_edu = ['Бак']*bac_q + ['Маг']*mag_q
    result['Уровень обучения'] = d_edu

    # Подразделения
    faculties = [text[i].contents[1].contents[1].contents[0]
                 for i in quantity]
    faculties.remove('Высшая школа бизнеса')  # Не открыта
    result['Подразделение'] = faculties

In [5]:
main()

**Перейдем к проверке наличия требуемых блоков на сайтах ОП**

### Общие разделы

In [6]:
# Функция для парсинга одной ссылки из одного блока

def pars_url(tag, attribute, soup):
    text = soup.find(tag, {"class": attribute})
    if text is None:
        url = 'Нет'
    else:
        url = text.contents[0].get('href')
        if len(url) is 0:
            url = 'Нет'
    return url

In [7]:
# Функция для парсинга нескольких ссылок из одного блока

def pars_url_list(tag, attribute, soup):
    text = soup.find(tag, {"class": attribute})
    if text is None:
        url = 'Нет'
    else:
        url = []
        for i in range(len(text)):
            text_2 = text.contents[i]
            while text_2.get('href') == None:
                text_2 = text_2.contents[0]
            url.append(text_2.get('href'))
    if len(url) is 0:
        url = 'Нет'
    return url

In [8]:
# Для парсинга языка обучения

def language(content):
    lang = []
    for i in range(len(content)):
        if len(content[i]) >= 2:
            lang.append('ENG_RUS')
        elif len(content[i]) == 1:
            lang.append(content[i][0].text)
        else:
            lang.append('Не указано')
    return lang

In [9]:
# Для определения на какой раздел указывают ссылки, спарсенные с помощью pars_url_list

def blocks(nav, part):
    block = []
    for i in range(len(nav)):
        b = 0
        for j in range(len(nav[i])):
            if (nav[i][j] == result['Ссылка на сайт'][i] + part):
                b = nav[i][j]
        if b == 0:
            block.append('Нет')
        else:
            block.append(b)
    return block

In [10]:
# Для парсинга текстов нескольких элементов из одного блока

def pars_text_list(tag, attribute, soup):
    text = soup.find(tag, {"class": attribute})
    if text is None:
        position = 'Нет'
    else:
        position = []
        for i in range(len(text)):
            position.append(text.contents[i].contents[1].text)
    if len(position) is 0:
        position = 'Нет'
    return position

In [11]:
# Для парсинга руководителей

def persons(pers_url, pers_name):
    ac_supervisor = []
    manager = []
    for i in range(len(pers_url)):
        if pers_url[i] == 'Нет':
            ac_supervisor.append('Нет')
            manager.append('Нет')
        else:
            a = 0
            m = 0
            for j in range(len(pers_name[i])):
                if (pers_name[i][j] == 'Академический руководитель') or (pers_name[i][j] == 'Руководитель'):
                    a = 'https:'+pers_url[i][j]
                if (pers_name[i][j] == 'Менеджер'):
                    m = 'https:'+pers_url[i][j]
            if a != 0:
                ac_supervisor.append(a)
            else:
                ac_supervisor.append('Нет')
            if m != 0:
                manager.append(m)
            else:
                manager.append('Нет')
    return [ac_supervisor, manager]

In [12]:
# Для парсинга учебного офиса

def st_office(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if text is None:
        url = 'Нет'
    else:
        index = 'no'
        for i in range(len(text)):
            if (text[i].text == 'Учебный офис'):
                index = i
                break
        if index == 'no':
            url = 'Нет'
        else:
            t = text[index]
            while t.get('href') == None:
                t = t.contents[0]
            url = t.get('href')
    if len(url) is 0:
        url = 'Нет'
    return url

In [13]:
# Для парсинга адреса

def address(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if text is None:
        return 'Нет'
    else:
        for i in range(len(text)):
            ad = re.search(
                r'(\d{6})|(бульвар)|(ул.)|(наб.)|(просп.)', text[i].text)
            if ad is None:
                continue
            else:
                return 'Есть адрес'
        return 'Нет'

In [14]:
# Телефон

def tel(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if text is None:
        return 'Нет'
    else:
        for i in range(len(text)):
            t = re.search(
                r'((8|\+7)[\- ]?)?(\(?\d{3}\)?[\- ]?)?[\d\- ]{7,10}', re.sub(r' ', '', text[i].text))
            if t is None:
                continue
            else:
                return t.group(0)
        return 'Нет'

In [15]:
# E-mail

def mail(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if text is None:
        return 'Нет'
    else:
        for i in range(len(text)):
            m = re.search(r'([a-z0-9_-]+\.)*[a-z0-9_-]+@hse.ru', text[i].text)
            if m is None:
                continue
            else:
                return m.group(0)
        return 'Нет'

**Основная функция для выгрузки общих разделов:**  

In [16]:
soups = []


def common_info():

    # Теперь найдем soup для всех ОП, чтобы далее к ним обращаться
    for url in result['Ссылка на сайт']:
        html = rq.get(url).text
        soups.append(soup_d(html))

    # Язык обучения
    content = [soup.findAll(
        "span", {"class": 'b-program__lang'}) for soup in soups]
    result['Язык обучения'] = language(content)

    navigation = [pars_url_list("ul", 'navigation icon_set1', soup) for soup in soups]

    result['Учебные курсы'] = blocks(navigation, part='courses')

    result['Преподаватели'] = blocks(navigation, part='tutors')

    result['Число студентов и вакантные места'] = blocks(navigation, part='vacant')

    result['Документы образовательной программы'] = blocks(navigation, part='documents')

    result['Расписание'] = blocks(navigation, part='timetable')

    result['День открытых дверей'] = [pars_url("div", 'g-day__desc', soup) for soup in soups]

    result['Академический совет'] = blocks(navigation, part='academ_council')

    # Академ руководитель и менеджер

    # Возьмем все ссылки и названия должностей
    pers_url = [pars_url_list("div", 'b-program__side-persons-group', soup) for soup in soups]
    pers_name = [pars_text_list("div", 'b-program__side-persons-group', soup) for soup in soups]
    pers = persons(pers_url, pers_name)
    result['Академический руководитель'] = pers[0]
    result['Менеджер'] = pers[1]
    # Одна программа указала Академ руководителя в другом блоке:
    index = result.loc[result['Ссылка на сайт'] ==
                       'https://www.hse.ru/ma/intercomp/'].index
    result.loc[index, 'Академический руководитель'] = 'https://www.hse.ru/org/persons/7529224'

    result['Учебный офис'] = [st_office("div", 'b-program__side-info with-indent4', soup) for soup in soups]

    result['Адрес'] = [address("div", 'b-program__side-info with-indent4', soup) for soup in soups]

    # Телефон (некоторые прораммы указывают телефон не в контактах, а как ссылку на
    # кнопку, дающую возможность звонить сразу с сайта, дополним список такими номерами)
    telephone = [tel("div", 'b-program__side-info with-indent4', soup) for soup in soups]
    dop_tel = [tel("a", 'button button_clean_red roistat-phone', soup) for soup in soups]
    for i in range(len(soups)):
        if (telephone[i] == 'Нет') and (dop_tel[i] != 'Нет'):
            telephone[i] = dop_tel[i]
    result['Телефон'] = telephone

    result['E-mail'] = [mail("div", 'b-program__side-info with-indent4', soup) for soup in soups]

    # Запишем файл
    result.to_excel("Общие разделы.xlsx")

In [17]:
common_info()

### Основные разделы

In [18]:
result_2 = pd.DataFrame()

Выгрузка **кратких реквизитов программы**

In [19]:
# Продолжительность обучения
def duration(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) > 0:
        duration = ' Нет'
        for i in range(len(text)):
            for j in range(len(text[i])):
                t = text[i].contents[j].text
                if re.search(r'(года)|(лет)', t) is None:
                    continue
                else:
                    duration = re.sub(r'[\n]+', '', t)
                    duration = re.sub(r' ', '', duration)
                    duration = re.sub(r'\xa0', ' ', duration)
                    break
    else:
        duration = ' Нет'
    return duration


# Форма обучения
def form(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) > 0:
        form = 'Нет'
        for i in range(len(text)):
            for j in range(len(text[i])):
                t = text[i].contents[j].text
                if re.search(r'форма', t) is None:
                    continue
                else:
                    form = re.search(
                        r'(очная)|(Очная)|(очно-заочная)|(Очно-заочная)|(заочная)|(Заочная)', t).group(0)
    else:
        form = 'Нет'
    return form


# Количество бюджетных и платных мест
def budget_paid(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) > 0:
        budget = 'Нет'
        paid = 'Нет'
        for i in range(len(text)):
            for j in range(len(text[i])):
                t = text[i].contents[j].text
                if re.search(r'мест', t) is None:
                    continue
                else:
                    t = re.sub(r'мест', '', t).split()
                    for i in range(len(t)):
                        if t[i] == 'бюджетных':
                            budget = re.sub(r'ВШЭ', '', t[i-1]) + ' мест'
                            break
                    for i in range(len(t)):
                        if t[i] == 'платных':
                            paid = re.sub(r'ВШЭ', '', t[i-1]) + ' мест'
                            break
    else:
        budget = 'Нет'
        paid = 'Нет'
    return [budget, paid]


# Аккредитация
def accred(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) > 0:
        for i in range(len(text)):
            accred = text[i].text
            if re.search(r'аккредитация', accred) is None:
                accred = 'Нет'
            else:
                accred = re.sub(r'\n', '', accred)
                # Создаем список на случай, если их несколько:
                accred = re.sub(r'аккредитация', '', accred)
    else:
        accred = 'Нет'
    return accred

In [20]:
# Базовая информация о программе (аннотация, количество знаков)

def annot(tag, attribute1, attribute2, soup):
    text = soup.findAll(tag, {"class": attribute1})
    if len(text) == 0:
        text = soup.findAll(tag, {"class": attribute2})
        if len(text) == 0:           
            return 'Нет'
        else:
            text = text[0].text
            length = len(text)
            return length
    else:
        text = text[0].text
        length = len(text)
        return length

In [21]:
# Партнеры

def partns(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) == 0:
        return 'Нет'
    else:
        partn = text[0].text
        partn = re.sub(r'(Университет–партнер)|(Университеты–партнеры)|(Программа создана совместно с)', '', partn)
    return partn

**Важные объявления**

В стандарте указано, что должно быть не более 3 объявлений, содержащих гиперссылку, а также не более 70 символов с пробелами. Соответсвенно, для каждой ОП создадим два столбца: 
- Список гиперссылок; 
- Список количеств символов в каждом объявлении, если более 3 объявлений, то "Больше 3х".

In [22]:
# Важные объявления

def announcement(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) == 0:
        urls = 'Нет'
        number_ann = 'Нет'
    else:
        urls = []
        number_ann = []
        for i in range(len(text)):
            url = text[i].contents[0].get('href')
            if (url is None) or (url == ''):
                urls.append('Нет')
            else:
                urls.append(url)
            number_ann.append(len(text[i].text))
        if len(number_ann) > 3:
            urls = 'Больше 3х'
            number_ann = 'Больше 3х'
    return [urls, number_ann]

In [23]:
# Текст о программе

def about(tag, attribute, soup):
    text = soup.findAll(tag, {"class": attribute})
    if len(text) == 0:
        return 'Нет'
    else:
        return len(text[0].text)

In [24]:
# Новости

def news(soup):
    text = soup.findAll("div", {"class": 'plate_news masonry'})
    if text is None:
        text = soup.findAll("div", {"class": 'post-meta__date'})
        if text is None:
            return 'Нет'
        else:
            return 'Есть'
    else:
        return 'Есть'

In [25]:
# Анонсы

def events(tag, attribute1, attribute2, soup):
    text = soup.findAll(tag, {"class": attribute1})
    if text is None:
        text = soup.findAll(tag, {"class": attribute2})
        if text is None:
            return 'Нет'
        else:
            return 'Есть'
    else:
        return 'Есть'

In [26]:
# Основная функция для выгрузки основных разделов:

soups_2 = []


def main_info():

    # Скопируем основную информацию из первой таблицы для быстрой навигации
    result_2[['Наименование ОП', 'Уровень обучения',
              'Подразделение', 'Ссылка на сайт']] = result[['Наименование ОП', 'Уровень обучения',
                                                            'Подразделение', 'Ссылка на сайт']]

    result_2['Продолжительность обучения'] = [duration("div", 'with-indent2 b-row__item b-row__item--size_3 b-row__item--t6', soup) for soup in soups]

    result_2['Форма обучения'] = [form("div", 'with-indent2 b-row__item b-row__item--size_3 b-row__item--t6', soup) for soup in soups]

    # Бюджетные и платные места
    places = [budget_paid("div", 'with-indent2 b-row__item b-row__item--size_3 b-row__item--t6', soup) for soup in soups]
    places = np.transpose(places)
    result_2['Бюджетные места'] = places[0]
    result_2['Платные места'] = places[1]

    result_2['Аккредитация'] = [accred("div", 'with-indent2 b-row__item b-row__item--size_3 b-row__item--t6', soup) for soup in soups]

    result_2['Аннотация (кол-во знаков)'] = [annot("div", 'with-indent lead-in _builder builder--text', 'g larger', soup) for soup in soups]
    
    result_2['Партнеры'] = [partns("div", 'b-stroke b-stroke--partners', soup) for soup in soups]

    # Важные объявления
    announc = [announcement("div", 'head-news__text', soup) for soup in soups]
    announc = np.transpose(announc)
    result_2['Ссылки на объявления'] = announc[0]
    result_2['Кол-во символов в объявлениях'] = announc[1]

    # Текст о программе
    urls = result['Ссылка на сайт']
    urls = urls + 'about/'
    for url in urls:
        html = rq.get(url).text
        soups_2.append(soup_d(html))
    result_2['Текст о программе (кол-во знаков)'] = [about("div", 'post__text', soup) for soup in soups_2]

    result_2['Новости'] = [news(soup) for soup in soups]

    result_2['Анонсы'] = [events("div", 'g-day__text', 'h5', soup) for soup in soups]

    # Запишем файл
    result_2.to_excel("Основные разделы.xlsx")

In [27]:
main_info()

### Раздел "Главное"

In [28]:
result_3 = pd.DataFrame()

In [29]:
def info(lst, newlst, links, linkss, v = 0):
    for i in lst:
        if i in links:
            for q in range(len(links)):
                if links[q] == i:
                    if linkss[q] in newlst:
                        pass
                    elif v == 1:
                        pass
                    else:
                        if linkss[q] is None:
                            newlst.append('Нет')
                        else:
                            newlst.append(linkss[q])
                        v = 1
                        break
        if i == lst[-1]:
            if v == 0:
                newlst.append('Нет')

In [30]:
infblock = []
about = []
partners = []
success = []
alumni = []
seminar = []
projects = []

In [31]:
def glavnoe():

    result_3[['Наименование ОП', 'Уровень обучения', 'Подразделение', 'Ссылка на сайт']] = result[[
        'Наименование ОП', 'Уровень обучения', 'Подразделение', 'Ссылка на сайт']]

    for z in range(len(soups)):
        block = soups[z].findAll('li', {'class': 'fa-sidemenu__item'})
        links = [i.contents[0].contents for i in block]
        linkss = [i.contents[0].get('href') for i in block]

        if soups[z].find('div', {'class': 'fa-sidemenu__header'}).contents[0].contents == ['О программе']:
            about.append(result_3['Ссылка на сайт'][z])
        else:
            about.append('Нет')

        partnerss = [['Партнеры '], ['Партнеры'], ['Партнеры и работодатели'], ['Наши партнеры'], [
            'Компании-партнеры'],  ['Эксперты - партнеры программы'], ['Попечительский совет и партнеры программы']]
        info(partnerss, partners, links, linkss)

        successs = [['Достижения студентов'], ['Достижения магистрантов']]
        info(successs, success, links, linkss)

        alumnis = [['Наши выпускники'], ['Выпускники']]
        info(alumnis, alumni, links, linkss)

        seminars = [['Научно-исследовательский семинар (НИС)'], ['НИС'], ['Научно-исследовательский семинар'], ['На\xadуч\xadно-ис\xadсле\xadдо\xadва\xadтель\xadский семинар'], [
            'Научный семинар'], ['Проектно-исследовательский семинар'], ['Проектный семинар'], ['Концепция научно- исследовательского семинара']]
        info(seminars, seminar, links, linkss)

        projectss = [['Проектная и исследовательская работа'], ['Проектная работа (на 2 и 3 курсе)'], ['Проектная работа'], ['Проектная деятельность'], ['Проектная деятельность'], ['Проекты'], ['Проекты и проектный семинар'], [
            'Проекты студентов'], ['Проекты и клубы студентов'], ['Проекты магистрантов'], ['Проекты и Проектный семинар'], ['Проекты и проектная работа'], ['Проекты и исследовательская работа'], ['Проекты и исследовательская деятельность']]
        info(projectss, projects, links, linkss)

    result_3['О программе'] = about
    result_3['Партнеры'] = partners
    result_3['Достижения студентов'] = success
    result_3['Выпускники'] = alumni
    result_3['НИС'] = seminar
    result_3['Проектная работа'] = projects

    # Запишем файл
    result_3.to_excel("Раздел 'Главное'.xlsx")

In [32]:
glavnoe()

### Раздел Абитуриентам

In [33]:
result_4 = pd.DataFrame()

In [34]:
future = []
adm = []
prepare = []
infblock2 = []
passport = []

In [35]:
soups_4 = []


def abit():

    result_4[['Наименование ОП', 'Уровень обучения', 'Подразделение']] = result[[
        'Наименование ОП', 'Уровень обучения', 'Подразделение']]

    urls_abit = [url + 'admission/' for url in result['Ссылка на сайт']]
    result_4['Ссылка на раздел абитуриентам'] = urls_abit

    # Теперь найдем soup
    for url in urls_abit:
        html = rq.get(url).text
        soups_4.append(soup_d(html))

    for z in range(len(soups_4)):
        block = soups_4[z].findAll(
            'a', {'class': 'link link_no-underline link_dark2 fa-sidemenu__link'})
        links = [i.contents for i in block]
        linkss = [i.get('href') for i in block]

        if links == []:
            infblock2.append('Нет')
        else:
            try:
                infblock2.append(soups_4[z].find('a', {
                                 'class': 'link link_no-underline link_dark2 selected fa-sidemenu__link selected'}).get('href'))
            except AttributeError:
                infblock2.append('Нет')

        try:
            if soups_4[z].find('a', {'class': 'link link_no-underline link_dark2 selected fa-sidemenu__link selected'}).contents == ['Паспорт программы']:
                passport.append(soups_4[z].find('a', {
                                'class': 'link link_no-underline link_dark2 selected fa-sidemenu__link selected'}).get('href'))
            else:
                passport.append('Нет')
        except AttributeError:
            try:
                if soups_4[z].find('h3').contents == ['Паспорт программы']:
                    passport.append(soups_4[z].find('a', {
                                    'class': 'link link_no-underline link_dark2 fa-sidemenu__link selected'}).get('href'))
                else:
                    passport.append('Нет')
            except AttributeError:
                passport.append('Нет')

        futuree = [['Будущая профессия'], ['Будущая профессия ']]
        info(futuree, future, links, linkss)

        admm = [['Траектории поступления'], ['Траектории поступления иностранных граждан'], [
            'Траектории поступления для иностранных абитуриентов']]
        info(admm, adm, links, linkss)

        preparee = [['Подготовка'], ['Подготовка к поступлению']]
        info(preparee, prepare, links, linkss)

    result_4['Информационный блок'] = infblock2
    result_4['Паспорт программы'] = passport
    result_4['Будущая профессия'] = future
    result_4['Траектории поступления'] = adm
    result_4['Подготовка'] = prepare

    # Запишем файл
    result_4.to_excel("Абитуриентам.xlsx")

In [36]:
abit()

### Раздел Студентам

In [37]:
result_5 = pd.DataFrame()

In [38]:
infblock3 = []
ratings = []
mobility = []
prac = []
diplomas = []
theses = []
assess = []
moocs = []
trc = []
inn = []

In [39]:
soups_5 = []


def students():

    # Скопируем основную информацию из первой таблицы для быстрой навигации
    result_5[['Наименование ОП', 'Уровень обучения', 'Подразделение']] = result[[
        'Наименование ОП', 'Уровень обучения', 'Подразделение']]

    urls_stud = [url + 'students/' for url in result['Ссылка на сайт']]
    result_5['Ссылка на раздел студентам'] = urls_stud

    # Теперь найдем soup Студентам
    for url in urls_stud:
        html = rq.get(url).text
        soups_5.append(soup_d(html))

    for i in range(len(soups_5)):
        block = soups_5[i].findAll(
            'a', {'class': 'link link_no-underline link_dark2 fa-sidemenu__link'})
        links = [j.contents for j in block]
        linkss = [k.get('href') for k in block]

        if links == []:
            infblock3.append('Нет')
        else:
            try:
                infblock3.append(soups_5[i].find('a', {
                    'link link_no-underline link_dark2 selected fa-sidemenu__link selected'}).get('href'))
            except AttributeError:
                infblock3.append('Нет')

        ratingss = [['Рейтинги'], ['Рейтинги '], [
            'Рейтинги студентов'], ['Рейтинг студентов'], ['Рейтинг']]
        info(ratingss, ratings, links, linkss)

        mobilityy = [['Студенческая мобильность'], ['Академическая мобильность'], ['Академическая мобильность студентов'], [
            'Международная академическая мобильность'], ['Внутриуниверситетская мобильность'], ['Международная мобильность']]
        info(mobilityy, mobility, links, linkss)

        pracs = [['Стажировки и практика'], ['Археологическая практика'], ['Музейная практика'], ['Архивная практика'], ['Учебная практика (1 курс)'], ['Производственная практика (3 курс)'], ['Преддипломная практика'], ['Проектная деятельность и преддипломная практика'], [
            'Производственная научно-исследовательская практика'], ['Практика и научно-исследовательская работа'], ['Практика'], ['Практика, проектная и исследовательская работа'], ['Практика, курсовые и ВКР'], ['Практика и проектная деятельность'],  ['Практика и проекты'], ['Практика 1 курса'], ['Практика и проекты 2 курса']]
        info(pracs, prac, links, linkss)

        diplomass = [['Каталог ВКР'], ['ВКР']]
        info(diplomass, diplomas, links, linkss)

        thesess = [['Практика, курсовые и ВКР'], ['Курсовые и ВКР'], ['Курсовые работы'], ['Курсовые'], [
            'Курсовые и выпускные квалификационные работы (ВКР)'], ['Курсовые работы и ВКР'], ['Курсовые и выпускные квалификационные работы'], ['Курсовые и проекты']]
        info(thesess, theses, links, linkss)

        assesss = [['Итоговая аттестация'], ['Итоговая аттестация '], ['Государственная итоговая аттестация'], ['Государственная итоговая аттестация '], [
            'ВКР и Итоговая аттестация'], ['Итоговая аттестация (Защита ВКР)'], ['Защита ВКР (Итоговая аттестация)']]
        info(assesss, assess, links, linkss)

        mooc = [['Рекомендованные онлайн-курсы'], ['Рекомендованные онлайн-курсы (MOOC)'], ['MOOC (онлайн курсы)'], [
            'Рекомендованные онлайн курсы'], ["MOOC's (онлайн курсы)"], ['Список рекомендованных онлайн курсов'], ['Курсы MOOC']]
        info(mooc, moocs, links, linkss)

        if ['Выбор траектории обучения'] in links:
            trc.append('https://electives.hse.ru')
        else:
            trc.append('Нет')

        if ['Дополнительные иностранные языки'] in links:
            inn.append('https://busedu.hse.ru/inn')
        else:
            inn.append('Нет')

    result_5['Информационный блок'] = infblock3
    result_5['Рейтинги'] = ratings
    result_5['Студенческая мобильность'] = mobility
    result_5['Практика'] = prac
    result_5['Каталог ВКР'] = diplomas
    result_5['Курсовые и ВКР'] = theses
    result_5['Итоговая аттестация'] = assess
    result_5['Рекомендованные онлайн-курсы'] = moocs
    result_5['Выбор траектории обучения'] = trc
    result_5['Дополнительные иностранные языки'] = inn

    # Запишем файл
    result_5.to_excel("Студентам.xlsx")

    print('Проверка всех пунктов Стандарта завершена')
    print("--- %s seconds ---" % (time.time() - start_time))

In [40]:
students()

Проверка всех пунктов Стандарта завершена
--- 2226.217045068741 seconds ---


**Подсчет количества знаков на всех страницах** (самая долгая функция)

In [57]:
headers = rq.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
 
def amount(url):
    if url == 'Нет':
        return 'Нет'
    else:
        html = rq.get(url, headers=headers).text
        soup = soup_d(html)
        return len(soup.text)

In [58]:
amount_counter = pd.DataFrame()
amount_counter[['Наименование ОП', 'Уровень обучения', 'Подразделение', 'Ссылка на сайт']] = result[[
        'Наименование ОП', 'Уровень обучения', 'Подразделение', 'Ссылка на сайт']]

In [59]:
start_time = time.time()

# Общие разделы
amount_counter['Учебные курсы'] = [amount(url) for url in result['Учебные курсы']]
amount_counter['Преподаватели'] = [amount(url) for url in result['Преподаватели']]
amount_counter['Число студентов и вакантные места'] = [amount(url) for url in result['Число студентов и вакантные места']]
amount_counter['Документы образовательной программы'] = [amount(url) for url in result['Документы образовательной программы']]
amount_counter['Расписание'] = [amount(url) for url in result['Расписание']]
amount_counter['День открытых дверей'] = [amount(url) for url in result['День открытых дверей']]
amount_counter['Академический совет'] = [amount(url) for url in result['Академический совет']]
amount_counter['Академический руководитель'] = [amount(url) for url in result['Академический руководитель']]
amount_counter['Менеджер'] = [amount(url) for url in result['Менеджер']]
amount_counter['Учебный офис'] = [amount(url) for url in result['Учебный офис']]

# Основные разделы
# Количество знаков "О программе" посчитано в Общих разделах, а в Разделе Главное есть ссылки

print("--- %s seconds ---" % (time.time() - start_time))

--- 3371.40043592453 seconds ---


In [60]:
start_time = time.time()

# Раздел Главное
amount_counter['Партнеры'] = [amount(url) for url in result_3['Партнеры']]
amount_counter['Достижения студентов'] = [amount(url) for url in result_3['Достижения студентов']]
amount_counter['Выпускники'] = [amount(url) for url in result_3['Выпускники']]
amount_counter['НИС'] = [amount(url) for url in result_3['НИС']]
amount_counter['Проектная работа'] = [amount(url) for url in result_3['Проектная работа']]

print("--- %s seconds ---" % (time.time() - start_time))

--- 1374.985915184021 seconds ---


In [61]:
start_time = time.time()

# Абитуриентам
amount_counter['Информационный блок Абитуриентам'] = [amount(url) for url in result_4['Информационный блок']]
amount_counter['Паспорт программы'] = [amount(url) for url in result_4['Паспорт программы']]
amount_counter['Будущая профессия'] = [amount(url) for url in result_4['Будущая профессия']]
amount_counter['Траектории поступления'] = [amount(url) for url in result_4['Траектории поступления']]
amount_counter['Подготовка'] = [amount(url) for url in result_4['Подготовка']]

print("--- %s seconds ---" % (time.time() - start_time))

--- 2115.44477891922 seconds ---


In [62]:
start_time = time.time()

# Студентам
amount_counter['Информационный блок Студентам'] = [amount(url) for url in result_5['Информационный блок']]
amount_counter['Рейтинги'] = [amount(url) for url in result_5['Рейтинги']]
amount_counter['Студенческая мобильность'] = [amount(url) for url in result_5['Студенческая мобильность']]
amount_counter['Практика'] = [amount(url) for url in result_5['Практика']]
amount_counter['Каталог ВКР'] = [amount(url) for url in result_5['Каталог ВКР']]
amount_counter['Курсовые и ВКР'] = [amount(url) for url in result_5['Курсовые и ВКР']]
amount_counter['Итоговая аттестация'] = [amount(url) for url in result_5['Итоговая аттестация']]
amount_counter['Рекомендованные онлайн-курсы'] = [amount(url) for url in result_5['Рекомендованные онлайн-курсы']]
amount_counter['Выбор траектории обучения'] = [amount(url) for url in result_5['Выбор траектории обучения']]
amount_counter['Дополнительные иностранные языки'] = [amount(url) for url in result_5['Дополнительные иностранные языки']]

print("--- %s seconds ---" % (time.time() - start_time))

--- 3075.8569781780243 seconds ---


In [63]:
amount_counter.to_excel('Количество знаков.xlsx')